In [1]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from sklearn.metrics import roc_auc_score,auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold,StratifiedGroupKFold
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(6)

polars.config.Config

In [3]:
path_to_train = "/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train"
path_to_test = "/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test"
path_to_features = "/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv"

In [4]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [5]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_1_0.parque…","""train_applprev_1_0.parquet"""
4,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_1_1.parque…","""train_applprev_1_1.parquet"""
24,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
21,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_base.parquet""","""train_base.parquet"""
29,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_credit_bureau_a_1_0…","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
18,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
26,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
14,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
27,"""/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_base.parquet""","""test_base.parquet"""


In [6]:
train_base = (
    pl.read_parquet(path_to_train+"/train_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
0,2019-01-03,1,1,4,0,0
1,2019-01-03,1,1,4,0,0
2,2019-01-04,1,1,5,0,0
…,…,…,…,…,…,…
"2,703,452",2020-10-05,10,41,1,1,0
"2,703,453",2020-10-05,10,41,1,1,0
"2,703,454",2020-10-05,10,41,1,1,0


In [7]:
test_base = (
    pl.read_parquet(path_to_test+"/test_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
test_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
"57,543",2021-05-14,5,19,5,2,0
"57,549",2022-01-17,1,3,1,3,0
"57,551",2020-11-27,11,48,5,1,0
…,…,…,…,…,…,…
"57,632",2022-02-05,2,5,6,3,0
"57,633",2022-01-25,1,4,2,3,0
"57,634",2021-01-27,1,4,3,2,0


In [8]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ],
        how="vertical"
    )
)

In [9]:
def reduce_dtypes(df:pl.DataFrame):
    return (
        df
        .with_columns(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("T","M").cast(pl.String),
            cs.ends_with("P","A").cast(pl.Float32),
            (cs.ends_with("L") & cs.numeric()).cast(pl.Float32),
        )
    )

def grouping(df):
    return (
        df
        .group_by("case_id")
        .agg(
            cs.numeric().max(),
            (~cs.numeric()).drop_nulls().mode().first()
        )
    )

def preprocess(filter_string:str,prefix_string:str):
    train_files_list = train_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    test_files_list = test_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    with pl.StringCache():
        train_df = (
            pl.concat(
                [
                    pl.read_parquet(_).pipe(reduce_dtypes).pipe(grouping) for _ in train_files_list
                ]
            )
        )
        test_df = (
            pl.concat(
                [
                    pl.read_parquet(_).select(train_df.columns).cast(train_df.schema).pipe(grouping) for _ in test_files_list
                ]
            )
        )
    return (
        pl.concat(
            [
                train_df,
                test_df
            ],
            how="vertical_relaxed"
        )
        .pipe(lambda df: df.rename({_:f"{prefix_string}_{_}" for _ in df.columns if not _ == 'case_id'}))
    )

def select_low_catcols(df:pl.DataFrame,thresh=200):
    col_names = []
    for col_name in df.select(cs.categorical()).columns:
        if df.select(pl.col(col_name).value_counts()).shape[0] > thresh:
            col_names.append(col_name)
    return df.select(~cs.by_name(col_names))
    

def select_impuatable(df:pl.DataFrame,thresh=0.95):
    cols =  (
        df
        .select(pl.all().is_null().mean())
        .transpose(include_header=True)
        .filter(pl.col("column_0") < thresh)
        ["column"].to_list()
    )
    return df.select(cols)

In [10]:
total_past_shallow = preprocess("applprev_1","past_shallow")
total_past_shallow

case_id,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_district_544M,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,past_shallow_isbidproduct_390L,past_shallow_isdebitcard_527L,past_shallow_postype_4733339M,past_shallow_profession_152M,past_shallow_rejectreason_755M,past_shallow_rejectreasonclient_4145042M,past_shallow_status_219L
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,date,str,date,str,str,date,str,date,date,str,date,str,date,str,bool,bool,str,str,str,str,str
"1,553,090",0.00,"3,624.00",null,4.00,null,0.00,null,null,null,"17,560.00",0.00,0.00,"36,000.00",0.00,1,0.00,6.00,null,6.00,2017-09-13,"""a55475b1""",2019-01-22,null,"""COL""",2017-09-19,"""P152_70_56""",2018-02-08,2018-02-08,"""a55475b1""",2014-03-15,"""MARRIED""",2017-10-15,"""POS""",false,null,"""a55475b1""","""a55475b1""","""P99_56_166""","""a55475b1""","""K"""
"2,573,820",0.00,"4,989.20","12,000.00",1.00,null,0.00,null,null,null,"60,000.00","30,113.48",0.00,"50,000.00",9.00,19,"35,112.00",36.00,null,36.00,2016-02-09,"""a55475b1""",2017-10-06,null,"""CAL""",2018-06-06,"""P188_113_3""",2018-11-24,2018-10-08,"""P97_36_170""",null,"""SINGLE""",2017-11-05,"""CASH""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"2,574,692",0.00,"7,331.60",null,2.00,null,0.00,null,null,null,"120,000.00",0.00,0.00,"42,000.00",18.00,2,0.00,36.00,null,36.00,2012-01-28,"""a55475b1""",2012-01-28,null,"""CAL""",2017-02-21,"""P158_121_167""",2018-04-11,2018-04-11,"""P97_36_170""",2011-09-15,"""MARRIED""",2017-03-20,"""CASH""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,549",0.00,"3,819.80","50,000.00",0.00,"34,066.00","34,066.00",0.00,0.00,0.00,"60,000.00","10,256.63",0.00,"60,000.00",1.00,13,"10,638.20",36.00,"840,049,792.00",36.00,2021-10-12,"""a55475b1""",2012-05-03,"""CL""","""REL""",2012-12-14,"""P158_150_171""",2021-10-13,2021-10-13,"""a55475b1""",1998-07-12,"""SINGLE""",2020-05-16,"""POS""",false,false,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
"57,760",0.00,"14,092.60",null,1.00,"100,000.00","100,000.00",0.00,0.00,0.00,"200,000.00",0.00,0.00,"112,000.00",18.00,7,0.00,24.00,"820,377,856.00",24.00,2020-06-15,"""a55475b1""",2020-06-15,"""AC""","""REL""",2020-08-07,"""P117_87_136""",2019-07-30,2020-06-15,"""a55475b1""",2009-02-12,"""MARRIED""",2016-03-12,"""NDF""",false,true,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"57,970",0.00,768.00,"18,000.00",2.00,null,"30,000.00",null,null,null,"30,000.00",0.00,0.00,"43,000.00",0.00,7,0.00,24.00,null,24.00,null,"""P94_109_143""",2021-04-02,null,"""REL""",null,"""a55475b1""",null,null,"""P97_36_170""",2006-12-30,"""MARRIED""",2012-10-11,"""CASH""",false,false,"""P177_117_192""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""


In [11]:
total_past_depth = preprocess("applprev_2","past_depth")
total_past_depth

case_id,past_depth_num_group1,past_depth_num_group2,past_depth_cacccardblochreas_147M,past_depth_conts_type_509L,past_depth_credacc_cards_status_52L
u32,u16,u16,str,str,str
"7,637",0,1,null,"""PRIMARY_MOBILE""",null
"21,330",0,1,null,"""PHONE""",null
"162,858",9,2,"""a55475b1""","""PRIMARY_MOBILE""",null
…,…,…,…,…,…
"2,541,935",2,1,"""a55475b1""","""PRIMARY_MOBILE""",null
"1,662,353",2,1,"""a55475b1""","""PRIMARY_MOBILE""",null
"57,543",4,1,"""a55475b1""","""PRIMARY_MOBILE""",null


In [12]:
total_static_base = preprocess("static_0","static_base")
total_static_base

case_id,static_base_actualdpdtolerance_344P,static_base_amtinstpaidbefduel24m_4187115A,static_base_annuity_780A,static_base_annuitynextmonth_57A,static_base_applicationcnt_361L,static_base_applications30d_658L,static_base_applicationscnt_1086L,static_base_applicationscnt_464L,static_base_applicationscnt_629L,static_base_applicationscnt_867L,static_base_avgdbddpdlast24m_3658932P,static_base_avgdbddpdlast3m_4187120P,static_base_avgdbdtollast24m_4525197P,static_base_avgdpdtolclosure24_3658938P,static_base_avginstallast24m_3658937A,static_base_avglnamtstart24m_4525187A,static_base_avgmaxdpdlast9m_3716943P,static_base_avgoutstandbalancel6m_4187114A,static_base_avgpmtlast12m_4525200A,static_base_clientscnt12m_3712952L,static_base_clientscnt3m_3712950L,static_base_clientscnt6m_3712949L,static_base_clientscnt_100L,static_base_clientscnt_1022L,static_base_clientscnt_1071L,static_base_clientscnt_1130L,static_base_clientscnt_136L,static_base_clientscnt_157L,static_base_clientscnt_257L,static_base_clientscnt_304L,static_base_clientscnt_360L,static_base_clientscnt_493L,static_base_clientscnt_533L,static_base_clientscnt_887L,static_base_clientscnt_946L,static_base_cntincpaycont9m_3716944L,…,static_base_credtype_322L,static_base_datefirstoffer_1144D,static_base_datelastinstal40dpd_247D,static_base_datelastunpaid_3546854D,static_base_disbursementtype_67L,static_base_dtlastpmtallstes_4499206D,static_base_equalitydataagreement_891L,static_base_equalityempfrom_62L,static_base_firstclxcampaign_1125D,static_base_firstdatedue_489D,static_base_inittransactioncode_186L,static_base_isbidproduct_1095L,static_base_isbidproductrequest_292L,static_base_isdebitcard_729L,static_base_lastactivateddate_801D,static_base_lastapplicationdate_877D,static_base_lastapprcommoditycat_1041M,static_base_lastapprcommoditytypec_5251766M,static_base_lastapprdate_640D,static_base_lastcancelreason_561M,static_base_lastdelinqdate_224D,static_base_lastrejectcommoditycat_161M,static_base_lastrejectcommodtypec_5251769M,static_base_lastrejectdate_50D,static_base_lastrejectreason_759M,static_base_lastrejectreasonclient_4145040M,static_base_lastrepayingdate_696D,static_base_lastst_736L,static_base_maxdpdinstldate_3546855D,static_base_opencred_647L,static_base_paytype1st_925L,static_base_paytype_783L,static_base_payvacationpostpone_4187118D,static_base_previouscontdistrict_112M,static_base_twobodfilling_608L,static_base_typesuite_864L,static_base_validfrom_1069D
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,str,date,date,date,str,date,bool,bool,date,date,str,bool,bool,bool,date,date,str,str,date,str,date,str,str,date,str,str,date,str,date,bool,str,str,date,str,str,str,date
"2,533,600",0.00,null,"4,978.20",0.00,0.00,0.00,0.00,0.00,0.00,4.00,null,null,null,1.00,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,…,"""CAL""",2007-10-29,null,2010-05-16,"""GBA""",null,null,null,2017-07-24,2008-09-12,"""CASH""",true,null,null,2009-08-21,2011-01-31,"""P12_6_178""","""a55475b1""",2009-08-18,"""a55475b1""",null,"""a55475b1""","""a55475b1""",2011-01-31,"""a55475b1""","""a55475b1""",null,"""D""",2008-12-11,false,"""OTHER""","""OTHER""",null,"""P157_94_145""","""FO""",null,2018-11-02
"858,867",0.00,"8,940.00","4,443.40","4,470.00",0.00,0.00,0.00,2.00,0.00,1.00,-1.00,0.00,0.00,0.00,"4,470.00","53,640.00",0.00,"46,890.00","4,470.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,…,"""COL""",null,null,2019-09-27,"""SBA""",2019-10-27,null,null,null,2019-08-27,"""POS""",false,null,null,2019-08-19,2019-07-27,"""P12_6_178""","""a55475b1""",2019-07-27,"""a55475b1""",2019-09-27,"""P21_79_33""","""a55475b1""",2016-12-02,"""P99_56_166""","""P94_109_143""",null,"""A""",2019-09-27,false,"""OTHER""","""OTHER""",null,"""P159_143_123""","""FO""",null,null
"162,155",0.00,"3,193.20","6,959.80",0.00,0.00,0.00,0.00,0

In [13]:
total_static_external = (
    preprocess("static_cb","static_external")
    .with_columns(
        pl.col("static_external_riskassesment_302T").str.split("%").list.gather([0,1]).apply(lambda x: (int(x[0])+int(x[1].split("-")[1]))/200)
    )
)
total_static_external

case_id,static_external_contractssum_5085716L,static_external_days120_123L,static_external_days180_256L,static_external_days30_165L,static_external_days360_512L,static_external_days90_310L,static_external_firstquarter_103L,static_external_for3years_128L,static_external_for3years_504L,static_external_for3years_584L,static_external_formonth_118L,static_external_formonth_206L,static_external_formonth_535L,static_external_forquarter_1017L,static_external_forquarter_462L,static_external_forquarter_634L,static_external_fortoday_1092L,static_external_forweek_1077L,static_external_forweek_528L,static_external_forweek_601L,static_external_foryear_618L,static_external_foryear_818L,static_external_foryear_850L,static_external_fourthquarter_440L,static_external_numberofqueries_373L,static_external_pmtaverage_3A,static_external_pmtaverage_4527227A,static_external_pmtaverage_4955615A,static_external_pmtcount_4527229L,static_external_pmtcount_4955617L,static_external_pmtcount_693L,static_external_pmtscount_423L,static_external_pmtssum_45A,static_external_secondquarter_766L,static_external_thirdquarter_1082L,static_external_assignmentdate_238D,static_external_assignmentdate_4527235D,static_external_assignmentdate_4955616D,static_external_birthdate_574D,static_external_dateofbirth_337D,static_external_dateofbirth_342D,static_external_description_5085714M,static_external_education_1103M,static_external_education_88M,static_external_maritalst_385M,static_external_maritalst_893M,static_external_requesttype_4525192L,static_external_responsedate_1012D,static_external_responsedate_4527233D,static_external_responsedate_4917613D,static_external_riskassesment_302T,static_external_riskassesment_940T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,date,date,date,date,date,date,str,str,str,str,str,str,date,date,date,f64,str
"119,976",null,3.00,4.00,3.00,5.00,3.00,2.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,5.00,null,null,null,null,null,null,7.00,"6,817.40",2.00,0.00,null,null,null,1968-02-01,1968-02-01,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,2019-04-09,null,null,null,null
"1,397,384",null,3.00,3.00,2.00,3.00,2.00,6.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,3.00,null,null,null,null,null,null,8.00,"42,544.03",2.00,1.00,null,null,null,1967-04-01,1967-04-01,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,2019-06-24,null,null,null,null
"1,443,648",null,2.00,6.00,2.00,8.00,2.00,6.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,8.00,null,null,null,null,null,null,8.00,"35,093.16",8.00,5.00,null,null,null,1966-10-01,1966-10-01,null,"""a55475b1""","""6b2ae0fa""","""a55475b1""","""3439d993""","""a55475b1""",null,2019-07-27,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,549","1,563,077.88",6.00,9.00,3.00,12.00,4.00,9.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.00,12.00,null,null,"26,815.60",null,14.00,null,null,null,8.00,2.00,null,null,2018-05-06,null,1959-11-01,null,"""2fc785b2""","""39a0853f""","""a55475b1""","""a7fcb6e5""","""a55475b1""",null,null,null,2022-01-31,null,null
"57,552","747,031.75",2.00,2.00,0.00,5.00,0.00,3.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.00,5.00,null,null,"23,402.80",null,14.00,null,null,null,7.00,1.00,null,null,2018-11-18,null,1955-11-01,null,"""2fc785b2""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,2020-12-11,null,null
"57,632","17,677.00",1.00,2.00,0.00,4.00,0.00,1.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,4.00,null,null,"15,841.20",null,14.00,null,null,null,1.00,1.00,null,null,2015-11-26,nul

In [14]:
total_person_shallow = preprocess("person_1","person_shallow")
total_person_shallow

case_id,person_shallow_childnum_185L,person_shallow_mainoccupationinc_384A,person_shallow_num_group1,person_shallow_personindex_1023L,person_shallow_persontype_1072L,person_shallow_persontype_792L,person_shallow_birth_259D,person_shallow_birthdate_87D,person_shallow_contaddr_district_15M,person_shallow_contaddr_matchlist_1032L,person_shallow_contaddr_smempladdr_334L,person_shallow_contaddr_zipcode_807M,person_shallow_education_927M,person_shallow_empl_employedfrom_271D,person_shallow_empl_employedtotal_800L,person_shallow_empl_industry_691L,person_shallow_empladdr_district_926M,person_shallow_empladdr_zipcode_114M,person_shallow_familystate_447L,person_shallow_gender_992L,person_shallow_housetype_905L,person_shallow_housingtype_772L,person_shallow_incometype_1044T,person_shallow_isreference_387L,person_shallow_language1_981M,person_shallow_maritalst_703L,person_shallow_registaddr_district_1083M,person_shallow_registaddr_zipcode_184M,person_shallow_relationshiptoclient_415T,person_shallow_relationshiptoclient_642T,person_shallow_remitter_829L,person_shallow_role_1084L,person_shallow_role_993L,person_shallow_safeguarantyflag_411L,person_shallow_sex_738L,person_shallow_type_25L
u32,f32,f32,u16,f32,f32,f32,date,date,str,bool,bool,str,str,date,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,bool,str,str,bool,str,str
"206,204",null,"74,000.00",1,0.00,1.00,1.00,1987-02-01,null,"""a55475b1""",false,false,"""P45_25_38""","""a55475b1""",2020-01-15,"""LESS_ONE""","""OTHER""","""P109_162_152""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""P109_162_152""","""P45_25_38""",null,null,null,"""CL""",null,true,"""M""","""PRIMARY_MOBILE"""
"2,655,555",null,"70,000.00",0,0.00,1.00,1.00,1974-11-01,null,"""P123_39_170""",false,false,"""P40_33_175""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,"""EMPLOYED""",null,"""P209_127_106""",null,"""P123_39_170""","""P40_33_175""",null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE"""
"2,661,412",null,"110,000.00",2,2.00,5.00,5.00,1956-03-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""","""WIDOWED""",null,null,null,"""RETIRED_PENSIONER""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""OTHER""","""CHILD""",false,"""PE""",null,true,"""M""","""PHONE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,551",null,"59,600.00",0,0.00,1.00,1.00,1982-05-01,null,"""P11_36_178""",false,false,"""P97_107_128""","""a55475b1""",2002-01-08,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",null,"""P11_36_178""","""P97_107_128""",null,null,null,"""CL""",null,false,"""F""","""PRIMARY_MOBILE"""
"57,569",0.00,"58,000.00",1,1.00,5.00,5.00,1949-09-01,1949-09-01,"""P121_131_159""",false,false,"""a55475b1""","""P106_81_188""",null,null,null,"""a55475b1""","""a55475b1""","""SINGLE""","""F""","""OWNED""","""OWNED""","""RETIRED_PENSIONER""",false,"""a55475b1""","""MARRIED""","""a55475b1""","""a55475b1""","""CHILD""","""CHILD""",false,"""PE""","""FULL""",false,"""F""","""PHONE"""
"57,549",null,"49,800.00",1,1.00,5.00,5.00,1959-11-01,null,"""a55475b1""",null,null,"""a55475b1""","""P106_81_188""",null,null,null,"""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""RETIRED_PENSIONER""",null,"""a55475b1""",null,"""a55475b1""","""P111_112_180""","""COLLEAGUE""","""COLLEAGUE""",false,"""CL""",null,true,"""F""","""PHONE"""


In [15]:
total_person_depth = preprocess("person_2","person_depth")
total_person_depth

case_id,person_depth_num_group1,person_depth_num_group2,person_depth_addres_district_368M,person_depth_addres_role_871L,person_depth_addres_zip_823M,person_depth_conts_role_79M,person_depth_empls_economicalst_849M,person_depth_empls_employedfrom_796D,person_depth_empls_employer_name_740M,person_depth_relatedpersons_role_762T
u32,u16,u16,str,str,str,str,str,date,str,str
"55,214",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"1,467,246",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"1,614,452",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
…,…,…,…,…,…,…,…,…,…,…
"57,551",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,569",1,5,"""P180_77_106""","""PERMANENT""","""P47_66_61""","""P38_92_157""","""a55475b1""",null,"""a55475b1""","""PARENT"""
"57,552",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null


In [16]:
total_other_shallow = preprocess("other_1","other_shallow")
total_other_shallow

case_id,other_shallow_amtdebitincoming_4809443A,other_shallow_amtdebitoutgoing_4809440A,other_shallow_amtdepositbalance_4809441A,other_shallow_amtdepositincoming_4809444A,other_shallow_amtdepositoutgoing_4809442A,other_shallow_num_group1
u32,f32,f32,f32,f32,f32,u16
"1,760,669",0.00,0.00,"11,483.40",0.00,"2,073.20",0
"219,569","33,333.40","33,333.40",0.00,0.00,0.00,0
"1,822,758",0.00,0.00,0.00,0.00,0.00,0
…,…,…,…,…,…,…
"57,648","15,233.40","15,233.40",0.00,0.00,0.00,0
"57,644",0.00,60.40,0.00,0.00,0.00,1
"57,650",0.00,0.00,234.60,0.00,1.40,0


In [17]:
total_deposit_shallow = preprocess("deposit_1","deposit_shallow")
total_deposit_shallow

case_id,deposit_shallow_amount_416A,deposit_shallow_num_group1,deposit_shallow_contractenddate_991D,deposit_shallow_openingdate_313D
u32,f32,u16,date,date
"1,860,227",0.00,0,null,2015-11-09
"2,640,615",202.01,0,2017-12-07,2015-12-08
"803,204",0.00,0,null,2013-11-07
…,…,…,…,…
"57,715","1,800.00",4,2016-03-16,2014-08-13
"57,644",0.00,2,2016-11-29,2013-08-08
"57,709",0.00,0,null,2015-09-14


In [18]:
total_debitcard_shallow = preprocess("debitcard","card_shallow")
total_debitcard_shallow

case_id,card_shallow_last180dayaveragebalance_704A,card_shallow_last180dayturnover_1134A,card_shallow_last30dayturnover_651A,card_shallow_num_group1,card_shallow_openingdate_857D
u32,f32,f32,f32,u16,date
"2,555,929",0.61,"78,000.00","78,000.00",0,null
"1,683,942",null,null,null,0,2015-10-19
"254,957",null,null,null,0,2017-03-29
…,…,…,…,…,…
"57,644",null,null,null,2,2013-11-29
"57,708",null,null,null,0,2016-04-21
"57,715",null,null,null,3,2015-11-09


In [19]:
total_credit_internal_shallow = preprocess("bureau_a_1","int_shallow")
total_credit_internal_shallow

case_id,int_shallow_annualeffectiverate_199L,int_shallow_annualeffectiverate_63L,int_shallow_contractsum_5085717L,int_shallow_credlmt_230A,int_shallow_credlmt_935A,int_shallow_debtoutstand_525A,int_shallow_debtoverdue_47A,int_shallow_dpdmax_139P,int_shallow_dpdmax_757P,int_shallow_instlamount_768A,int_shallow_instlamount_852A,int_shallow_interestrate_508L,int_shallow_monthlyinstlamount_332A,int_shallow_monthlyinstlamount_674A,int_shallow_nominalrate_281L,int_shallow_nominalrate_498L,int_shallow_num_group1,int_shallow_numberofcontrsvalue_258L,int_shallow_numberofcontrsvalue_358L,int_shallow_numberofinstls_229L,int_shallow_numberofinstls_320L,int_shallow_numberofoutstandinstls_520L,int_shallow_numberofoutstandinstls_59L,int_shallow_numberofoverdueinstlmax_1039L,int_shallow_numberofoverdueinstlmax_1151L,int_shallow_numberofoverdueinstls_725L,int_shallow_numberofoverdueinstls_834L,int_shallow_outstandingamount_354A,int_shallow_outstandingamount_362A,int_shallow_overdueamount_31A,int_shallow_overdueamount_659A,int_shallow_overdueamountmax2_14A,int_shallow_overdueamountmax2_398A,int_shallow_overdueamountmax_155A,int_shallow_overdueamountmax_35A,int_shallow_periodicityofpmts_1102L,…,int_shallow_totalamount_6A,int_shallow_totalamount_996A,int_shallow_totaldebtoverduevalue_178A,int_shallow_totaldebtoverduevalue_718A,int_shallow_totaloutstanddebtvalue_39A,int_shallow_totaloutstanddebtvalue_668A,int_shallow_classificationofcontr_13M,int_shallow_classificationofcontr_400M,int_shallow_contractst_545M,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_financialinstitution_591M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_426M,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,int_shallow_subjectrole_182M,int_shallow_subjectrole_93M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,str,str,str,str,date,date,date,date,date,str,str,str,str,str,str,str,date,date,date,date,date,date,str,str,str,str,str,str,date,str,str
"1,345,999",null,null,null,null,"30,000.00","75,385.44",0.00,0.00,null,"2,260.00",null,null,"4,107.65",null,0.00,null,36,3.00,34.00,null,18.00,null,8.00,11.00,null,0.00,null,null,"27,580.65",null,0.00,"3,218.80",null,0.00,null,null,…,null,"52,000.00",0.00,0.00,"75,385.44",0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2019-10-26,null,null,2012-12-15,null,"""a55475b1""",null,"""6.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-04-30,null,null,2014-10-26,null,2014-10-26,null,"""7.0""","""2018.0""",null,"""a55475b1""","""a55475b1""",2019-05-04,"""a55475b1""","""a55475b1"""
"1,302,283",null,null,null,null,"60,000.00","47,938.47",0.00,20.00,null,"3,936.80",null,null,"3,936.80",null,null,null,10,2.00,4.00,null,null,null,null,22.00,null,0.00,null,null,null,null,0.00,554.50,null,554.50,null,null,…,null,null,0.00,0.00,"47,938.47",0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2020-07-06,null,null,2017-07-06,null,"""a55475b1""",null,"""3.0""","""2019.0""",null,"""a55475b1""","""a55475b1""",2019-03-12,null,null,2019-03-07,null,2019-03-07,null,"""12.0""","""2019.0""",null,"""a55475b1""","""a55475b1""",2019-03-21,"""a55475b1""","""a55475b1"""
"1,246,209",nul

In [20]:
total_credit_internal_depth = preprocess("bureau_a_2","int_depth")
total_credit_internal_depth

case_id,int_depth_collater_valueofguarantee_1124L,int_depth_collater_valueofguarantee_876L,int_depth_num_group1,int_depth_num_group2,int_depth_pmts_dpd_1073P,int_depth_pmts_dpd_303P,int_depth_pmts_overdue_1140A,int_depth_pmts_overdue_1152A,int_depth_collater_typofvalofguarant_298M,int_depth_collater_typofvalofguarant_407M,int_depth_collaterals_typeofguarante_359M,int_depth_collaterals_typeofguarante_669M,int_depth_pmts_month_158T,int_depth_pmts_month_706T,int_depth_pmts_year_1139T,int_depth_pmts_year_507T,int_depth_subjectroles_name_541M,int_depth_subjectroles_name_838M
u32,f32,f32,u16,u16,f32,f32,f32,f32,str,str,str,str,str,str,str,str,str,str
"619,231",0.00,null,1,35,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""12.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
"1,277,417",0.00,null,0,23,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""3.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
"637,068",0.00,null,0,35,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""7.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,633","7,230,000.00",0.00,8,1,0.00,0.00,0.00,0.00,"""9a0c095e""","""9a0c095e""","""c7a5ad39""","""c7a5ad39""","""2.0""","""2.0""","""2021.0""","""2021.0""","""ab3c25cf""","""ab3c25cf"""
"57,645",0.00,0.00,9,0,null,null,null,null,"""a55475b1""","""8fd95e4b""","""c7a5ad39""","""a55475b1""","""2.0""","""2.0""","""2020.0""","""2019.0""","""ab3c25cf""","""a55475b1"""
"57,549",0.00,0.00,9,0,null,null,null,null,"""a55475b1""","""9a0c095e""","""c7a5ad39""","""a55475b1""","""2.0""","""2.0""","""2022.0""","""2019.0""","""ab3c25cf""","""a55475b1"""


In [21]:
total_credit_external_shallow = preprocess("bureau_b_1","ext_shallow")
total_credit_external_shallow

case_id,ext_shallow_amount_1115A,ext_shallow_credlmt_1052A,ext_shallow_credlmt_228A,ext_shallow_credlmt_3940954A,ext_shallow_credquantity_1099L,ext_shallow_credquantity_984L,ext_shallow_debtpastduevalue_732A,ext_shallow_debtvalue_227A,ext_shallow_dpd_550P,ext_shallow_dpd_733P,ext_shallow_dpdmax_851P,ext_shallow_installmentamount_644A,ext_shallow_installmentamount_833A,ext_shallow_instlamount_892A,ext_shallow_interesteffectiverate_369L,ext_shallow_interestrateyearly_538L,ext_shallow_maxdebtpduevalodued_3940955A,ext_shallow_num_group1,ext_shallow_numberofinstls_810L,ext_shallow_overdueamountmax_950A,ext_shallow_pmtdaysoverdue_1135P,ext_shallow_pmtnumpending_403L,ext_shallow_residualamount_1093A,ext_shallow_residualamount_127A,ext_shallow_residualamount_3940956A,ext_shallow_totalamount_503A,ext_shallow_totalamount_881A,ext_shallow_classificationofcontr_1114M,ext_shallow_contractdate_551D,ext_shallow_contractmaturitydate_151D,ext_shallow_contractst_516M,ext_shallow_contracttype_653M,ext_shallow_credor_3940957M,ext_shallow_dpdmaxdatemonth_804T,ext_shallow_dpdmaxdateyear_742T,ext_shallow_lastupdate_260D,ext_shallow_overdueamountmaxdatemonth_494T,ext_shallow_overdueamountmaxdateyear_432T,ext_shallow_periodicityofpmts_997L,ext_shallow_periodicityofpmts_997M,ext_shallow_pmtmethod_731M,ext_shallow_purposeofcred_722M,ext_shallow_subjectrole_326M,ext_shallow_subjectrole_43M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,date,date,str,str,str,str,str,date,str,str,str,str,str,str,str,str
"1,421,458",null,"644,293.62",0.00,"644,293.62",1.00,4.00,0.00,null,0.00,0.00,0.00,0.00,"358,634.84",null,null,null,0.00,1,null,0.00,0.00,null,0.00,"358,634.84","358,634.84","644,293.62","114,135.60","""ea6782cc""",2017-10-26,2019-10-26,"""7241344e""","""a55475b1""","""a55475b1""","""11.0""","""2017.0""",2019-07-05,"""11.0""","""2017.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""a55475b1""","""ab3c25cf"""
"229,603","41,442.60",null,"35,600.00",null,3.00,14.00,0.00,"31,081.80",0.00,0.00,0.00,0.00,"47,775.34","3,897.40",null,42.00,0.00,2,12.00,0.00,0.00,9.00,0.00,null,null,"89,504.40","590,856.00","""ea6782cc""",2019-08-21,2020-09-04,"""7241344e""","""4257cbed""","""P0_31_66""","""4.0""","""2020.0""",2020-06-04,"""4.0""","""2020.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""a55475b1""","""ab3c25cf"""
"1,851,673","120,000.00",0.00,null,0.00,1.00,9.00,0.00,"59,117.26",0.00,0.00,0.00,0.00,"59,117.26","4,459.63",null,null,0.00,1,36.00,0.00,0.00,15.00,null,0.00,0.00,"120,000.00","554,185.44","""ea6782cc""",2015-10-05,2021-10-05,"""7241344e""","""1c9c5356""","""c5a72b57""","""11.0""","""2015.0""",2020-06-09,"""9.0""","""2018.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,775","3,000.00",null,null,null,null,8.00,0.00,0.00,null,0.00,"17,168.00",0.00,null,0.00,30.00,2.00,1.60,6,1.00,1.60,0.00,0.00,null,null,null,null,"156,400.00","""a55475b1""",2021-01-20,2020-07-25,"""7241344e""","""4257cbed""","""3e0bc39b""","""10.0""","""2019.0""",2021-12-01,"""4.0""","""2019.0""",null,"""3ecc50a0""","""dbcbe8f8""","""96a8fdfe""","""a55475b1""","""a55475b1"""
"57,754","1,932,619.38","132,032.00","38,000.00","76,800.00",4.00,11.00,0.00,"1,473,622.38",0.00,0.00,0.00,0.00,"1,473,622.38","8,052.58",null,39.00,0.00,4,240.00,0.00,0.00,183.00,0.00,"68,059.91","50,039.51","1,932,619.38","575,772.44","""ea6782cc""",2020-09-15,2024-12-25,"""7241344e""","""1c9c5356""","""b619fa46""","""11.0""","""2019.0""",2022-06-04,"""11.0""","""2019.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""a55475b1""","""a55475b1"""
"57,675","1,488,000.00","220,598.00","10,000.00","200,598.00",2.00,19.00,0.00,"1,433,179.00",0.00,0.00,"37,403.00",0.00,"1,433,179.00","17,030.26",null,null,0.00,2,181.00,10.60,0.00,160.00,0.00,"182,816.69","165,095.89","1,488,000.00","480,262.59","""ea6782

In [22]:
total_credit_external_depth = preprocess("bureau_b_2","ext_depth")
total_credit_external_depth

case_id,ext_depth_num_group1,ext_depth_num_group2,ext_depth_pmts_dpdvalue_108P,ext_depth_pmts_pmtsoverdue_635A,ext_depth_pmts_date_1107D
u32,u16,u16,f32,f32,date
"1,689,292",1,35,0.00,0.00,2019-11-15
"1,615,720",0,3,0.00,0.00,2019-09-15
"930,173",0,35,"858,276.00",114.00,2018-02-15
…,…,…,…,…,…
"687,732",1,9,"23,417.00",0.20,2019-03-15
"1,867,827",0,14,"9,195.00",4.60,2019-08-15
"57,675",0,9,0.00,0.00,2020-10-14


In [23]:
total_registry_a = preprocess("registry_a","reg_a")
total_registry_a

case_id,reg_a_amount_4527230A,reg_a_num_group1,reg_a_name_4527232M,reg_a_recorddate_4527225D
u32,f32,u16,str,date
"955,665","3,189.60",6,"""afab7913""",2020-02-25
"1,553,894","9,612.40",5,"""dbcf3d3e""",2019-10-10
"836,097",850.00,1,"""7ae96eeb""",2019-11-03
…,…,…,…,…
"1,623,243","6,091.40",5,"""595fef9d""",2019-11-24
"57,754","3,807.80",7,"""cc68a90a""",2022-06-16
"57,675","5,061.80",5,"""ff9eb829""",2022-01-07


In [24]:
total_registry_b = preprocess("registry_b","reg_b")
total_registry_b

case_id,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_b_name_4917606M
u32,f32,u16,date,str
"1,850,637","8,169.60",6,2020-01-19,"""5e180ef0"""
"1,884,471","20,020.60",5,2020-05-04,"""9295b78d"""
"236,602","11,469.60",4,2020-05-14,"""ae60481e"""
…,…,…,…,…
"253,465","9,942.00",3,2020-07-08,"""aa2a2b44"""
"57,543","27,191.80",5,2021-05-19,"""787c689d"""
"57,551","14,644.80",6,2020-07-30,"""d580dfef"""


In [25]:
total_registry_c = preprocess("registry_c","reg_c")
total_registry_c

case_id,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_employername_160M,reg_c_processingdate_168D
u32,u16,f32,str,date
"1,291,940",5,"1,300.00","""c22b2de9""",2018-11-20
"703,140",5,"25,701.77","""8ecedd6c""",2019-06-04
"1,470,592",17,"4,309.20","""6d5a6e92""",2019-05-31
…,…,…,…,…
"1,248,200",5,"2,374.80","""ea84b749""",2019-01-03
"1,540,123",6,"3,439.40","""3ba03bdd""",2019-09-30
"117,224",12,"4,989.63","""5e180ef0""",2018-11-16


In [26]:
gain_df = pl.read_csv("/kaggle/input/total-df/gains.csv")
drop_list = gain_df.filter(pl.col("gain") < 1000)["col_name"].to_list() + ["Date"]

In [27]:
total_df = (
    total_base
    .join(
        total_past_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_external,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .with_columns(
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days(),
        pl.col(pl.String).cast(pl.Categorical)
    )
    .drop(drop_list)
    .pipe(select_impuatable)
    .pipe(select_low_catcols)
    .with_columns(
        pl.col(pl.INTEGER_DTYPES).fill_null(strategy="mean"),
        pl.col(pl.FLOAT_DTYPES).fill_null(strategy="mean"),
        pl.col(pl.Boolean).fill_null(value=pl.col(pl.Boolean).drop_nulls().mode().first()),
        pl.col(pl.Categorical).fill_null(value=pl.col(pl.Categorical).drop_nulls().mode().first())
    )
    .with_columns(
        pl.col(pl.Categorical).to_physical()
    )
    .with_columns(
        (pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]) - pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).min())/(pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).max()-pl.col(pl.INTEGER_DTYPES).exclude(["case_id_base","target"]).min()),
        (pl.col(pl.FLOAT_DTYPES) - pl.col(pl.FLOAT_DTYPES).mean())/(pl.col(pl.FLOAT_DTYPES).std())
    )
    .select(
        ~cs.by_name("target"),
        cs.by_name("target")
    )
)
total_df

case_id_base,Date,month,week,weekday,year,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,…,int_shallow_classificationofcontr_13M,int_shallow_classificationofcontr_400M,int_shallow_contractst_545M,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,reg_a_amount_4527230A,reg_a_num_group1,reg_a_recorddate_4527225D,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_processingdate_168D,target
u32,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f64,f64,f32,f64,f64,f64,f32,f64,u8
0,NaN,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.79,0.00,0.79,0.20,0.50,0.79,0.89,0.92,0.20,0.83,0.25,0.77,…,0.00,0.00,0.00,0.00,0.18,0.59,0.75,0.90,0.59,0.00,0.00,0.18,0.29,0.06,0.00,1.00,0.78,0.71,0.88,0.69,0.88,0.64,0.36,0.29,0.06,0.00,0.99,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.60,0
1,NaN,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.79,0.00,0.79,0.20,0.50,0.79,0.89,0.92,0.20,0.83,0.25,0.77,…,0.00,0.00,0.00,0.00,0.18,0.59,0.75,0.90,0.59,0.00,0.00,0.18,0.29,0.06,0.00,1.00,0.78,0.71,0.88,0.69,0.88,0.64,0.36,0.29,0.06,0.00,0.99,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.60,0
2,NaN,0.00,0.00,0.67,0.00,-0.01,-1.08,0.00,-1.12,0.00,-0.45,0.00,0.00,0.00,-1.06,0.00,-0.35,-1.58,0.00,0.05,0.00,0.08,0.00,0.08,0.79,0.00,0.61,0.20,0.00,0.79,0.89,0.92,0.00,0.85,0.00,0.61,…,0.00,0.00,0.00,0.00,0.18,0.59,0.75,0.90,0.59,0.00,0.00,0.18,0.29,0.06,0.00,1.00,0.78,0.71,0.88,0.69,0.88,0.64,0.36,0.29,0.06,0.00,0.99,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.60,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,632",NaN,0.09,0.08,0.83,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.79,0.00,0.79,0.20,0.50,0.79,0.89,0.92,0.20,0.83,0.25,0.77,…,0.00,0.00,0.00,0.00,0.18,0.59,0.75,0.90,0.59,0.00,0.00,0.18,0.29,0.06,0.00,1.00,0.78,0.7

In [28]:
submission_df = total_df.filter(pl.col("case_id_base").is_in(test_base.select("case_id_base"))).drop("target")
submission_df.head()

case_id_base,Date,month,week,weekday,year,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,…,int_shallow_classificationofcontr_13M,int_shallow_classificationofcontr_400M,int_shallow_contractst_545M,int_shallow_contractst_964M,int_shallow_dateofcredend_289D,int_shallow_dateofcredend_353D,int_shallow_dateofcredstart_181D,int_shallow_dateofcredstart_739D,int_shallow_dateofrealrepmt_138D,int_shallow_description_351M,int_shallow_dpdmaxdatemonth_442T,int_shallow_dpdmaxdatemonth_89T,int_shallow_dpdmaxdateyear_596T,int_shallow_dpdmaxdateyear_896T,int_shallow_financialinstitution_382M,int_shallow_lastupdate_1112D,int_shallow_lastupdate_388D,int_shallow_numberofoverdueinstlmaxdat_148D,int_shallow_numberofoverdueinstlmaxdat_641D,int_shallow_overdueamountmax2date_1002D,int_shallow_overdueamountmax2date_1142D,int_shallow_overdueamountmaxdatemonth_284T,int_shallow_overdueamountmaxdatemonth_365T,int_shallow_overdueamountmaxdateyear_2T,int_shallow_overdueamountmaxdateyear_994T,int_shallow_purposeofcred_874M,int_shallow_refreshdate_3813885D,reg_a_amount_4527230A,reg_a_num_group1,reg_a_recorddate_4527225D,reg_b_amount_4917619A,reg_b_num_group1,reg_b_deductiondate_4917603D,reg_c_num_group1,reg_c_pmtamount_36A,reg_c_processingdate_168D,target
u32,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f64,f64,f32,f64,f64,f64,f32,f64,u8
"57,543",NaN,0.36,0.35,0.67,0.67,-0.01,-0.14,-0.54,0.56,0.00,-0.45,0.00,0.00,0.00,0.62,-0.22,0.48,-0.69,0.04,0.47,-0.28,1.40,0.00,1.40,0.97,0.00,0.61,0.20,0.50,0.99,0.93,0.93,0.60,0.83,0.25,0.61,…,0.00,0.01,0.00,0.04,0.18,0.56,0.94,0.96,0.60,0.00,1.00,0.18,0.57,0.94,0.50,1.00,0.91,0.66,0.88,0.65,0.88,0.45,0.18,0.43,0.94,0.17,1.00,0.00,0.06,0.91,-0.69,0.05,0.96,0.04,0.00,0.60,0
"57,549",NaN,0.00,0.04,0.00,1.00,-0.01,-0.50,0.99,-1.12,1.21,1.01,0.18,0.84,-0.46,-0.22,-0.28,-0.35,-0.13,-0.20,0.68,-0.32,1.14,4.18,1.14,0.98,0.00,0.34,0.00,1.00,0.38,0.97,0.97,0.20,0.59,0.00,0.88,…,0.00,0.02,0.00,0.04,0.17,0.59,0.85,0.75,0.58,0.00,0.45,0.55,1.00,0.29,0.21,1.00,0.89,0.69,0.88,0.66,0.88,0.27,0.55,1.00,0.76,0.06,1.00,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.60,0
"57,551",NaN,0.91,0.92,0.67,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.79,0.00,0.79,0.20,0.50,0.79,0.89,0.92,0.20,0.83,0.25,0.77,…,0.00,0.01,0.00,0.04,0.22,0.60,0.34,0.93,0.56,0.00,0.55,0.00,0.57,0.06,0.03,1.00,0.97,0.87,0.88,0.94,0.88,1.00,0.00,0.43,0.18,0.11,1.00,0.00,0.06,0.91,-1.68,0.06,0.31,0.04,0.00,0.60,0
"57,552",NaN,0.91,0.92,0.67,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.79,0.00,0.79,0.20,0.50,0.79,0.89,0.92,0.20,0.83,0.25,0.77,…,0.00,0.00,0.00,0.00,0.18,0.59,0.75,0.90,0.59,0.00,0.00,0.18,0.29,0.06,0.00,1.00,0.78,0.71,0.88,0.69,0.88,0.64,0.36,0.29,0.06,0.00,0.99,0.00,0.06,0.91,0.00,0.06,0.55,0.04,0.00,0.60,0
"57,569",NaN,1.00,0.98,

In [29]:
del total_df
del total_past_shallow
del total_past_depth
del total_static_base
del total_static_external
del total_person_depth
del total_person_shallow
del total_other_shallow
del total_deposit_shallow
del total_debitcard_shallow
del total_credit_external_depth
del total_credit_external_shallow
del total_credit_internal_depth
del total_credit_internal_shallow
del total_registry_a
del total_registry_b
del total_registry_c
gc.collect()

In [30]:
model = lgb.Booster(model_file="/kaggle/input/model_863447/other/model_863447/1/my_model.txt")

In [ ]:
sub_df = pd.DataFrame({
    "case_id": submission_df['case_id_base'].to_list(),
    "score": model.predict(submission_df.drop("case_id_base"))
}).set_index("case_id")
sub_df.to_csv("./submission.csv")